# EMT Simulation of topology with slack, line and PQ load

In [ ]:
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import matplotlib.pyplot as plt
import dpsimpy
import re

# %matplotlib widget

## Simulation

In [ ]:
# Parameters
V_nom = 20e3
p_load_nom = 100e3
q_load_nom = 50e3
line_resistance = 0.05
line_inductance = 0.1
line_capacitance = 0.1e-6

# Simulation parameters
time_step = 0.001
final_time = 1.0

# POWERFLOW FOR INITIALIZATION
time_step_pf = final_time
final_time_pf = final_time + time_step_pf
sim_name_pf = "EMT_Slack_PiLine_PQLoad_with_PF_Init_PF"
dpsimpy.Logger.set_log_dir("logs/" + sim_name_pf)

# Components
n1_pf = dpsimpy.sp.SimNode("n1", dpsimpy.PhaseType.Single)
n2_pf = dpsimpy.sp.SimNode("n2", dpsimpy.PhaseType.Single)

extnet_pf = dpsimpy.sp.ph1.NetworkInjection("Slack", dpsimpy.LogLevel.debug)
extnet_pf.set_parameters(voltage_set_point=V_nom)
extnet_pf.set_base_voltage(V_nom)
extnet_pf.modify_power_flow_bus_type(dpsimpy.PowerflowBusType.VD)

line_pf = dpsimpy.sp.ph1.PiLine("PiLine", dpsimpy.LogLevel.debug)
line_pf.set_parameters(R=line_resistance, L=line_inductance, C=line_capacitance)
line_pf.set_base_voltage(V_nom)

load_pf = dpsimpy.sp.ph1.Shunt("Load", dpsimpy.LogLevel.debug)
load_pf.set_parameters(G=p_load_nom / (V_nom**2), B=-q_load_nom / (V_nom**2))
load_pf.set_base_voltage(V_nom)

# Topology
extnet_pf.connect([n1_pf])
line_pf.connect([n1_pf, n2_pf])
load_pf.connect([n2_pf])
system_pf = dpsimpy.SystemTopology(50, [n1_pf, n2_pf], [extnet_pf, line_pf, load_pf])

# Logging
logger_pf = dpsimpy.Logger(sim_name_pf)
logger_pf.log_attribute("v1", "v", n1_pf)
logger_pf.log_attribute("v2", "v", n2_pf)
logger_pf.log_attribute("i12", "i_intf", line_pf)

# Simulation
sim_pf = dpsimpy.Simulation(sim_name_pf, dpsimpy.LogLevel.debug)
sim_pf.set_system(system_pf)
sim_pf.set_time_step(time_step_pf)
sim_pf.set_final_time(final_time_pf)
sim_pf.set_domain(dpsimpy.Domain.SP)
sim_pf.set_solver(dpsimpy.Solver.NRP)
sim_pf.do_init_from_nodes_and_terminals(False)
sim_pf.add_logger(logger_pf)
sim_pf.run()


# DYNAMIC SIMULATION
time_step_emt = time_step
final_time_emt = final_time + time_step_emt
sim_name_emt = "EMT_Slack_PiLine_PQLoad_with_PF_Init_EMT"
dpsimpy.Logger.set_log_dir("logs/" + sim_name_emt)

# Components
gnd = dpsimpy.emt.SimNode.gnd
n1_emt = dpsimpy.emt.SimNode("n1", dpsimpy.PhaseType.ABC)
n2_emt = dpsimpy.emt.SimNode("n2", dpsimpy.PhaseType.ABC)

extnet_emt = dpsimpy.emt.ph3.NetworkInjection("Slack", dpsimpy.LogLevel.debug)
line_emt = dpsimpy.emt.ph3.PiLine("PiLine", dpsimpy.LogLevel.debug)

line_emt.set_parameters(
    dpsimpy.Math.single_phase_parameter_to_three_phase(line_resistance),
    dpsimpy.Math.single_phase_parameter_to_three_phase(line_inductance),
    dpsimpy.Math.single_phase_parameter_to_three_phase(line_capacitance),
)

load_emt = dpsimpy.emt.ph3.RXLoad("Load", dpsimpy.LogLevel.debug)
load_emt.set_parameters(
    dpsimpy.Math.single_phase_power_to_three_phase(p_load_nom),
    dpsimpy.Math.single_phase_power_to_three_phase(q_load_nom),
    V_nom,
)


# Topology
extnet_emt.connect([n1_emt])
line_emt.connect([n1_emt, n2_emt])
load_emt.connect([n2_emt])
system_emt = dpsimpy.SystemTopology(
    50, [n1_emt, n2_emt], [extnet_emt, line_emt, load_emt]
)

# Initialization of dynamic topology
system_emt.init_with_powerflow(system_pf, dpsimpy.Domain.EMT)

# Logging
logger_emt = dpsimpy.Logger(sim_name_emt)
logger_emt.log_attribute("v1", "v", n1_emt)
logger_emt.log_attribute("v2", "v", n2_emt)
logger_emt.log_attribute("i12", "i_intf", line_emt)
logger_emt.log_attribute("irx", "i_intf", load_emt)

# load step sized in absolute terms
load_switch = dpsimpy.emt.ph3.Switch("Load_Add_Switch_n2", dpsimpy.LogLevel.debug)
connection_node = system_emt.node("n2")
resistance = np.abs(connection_node.initial_single_voltage()) ** 2 / 100e3
load_switch.set_parameters(np.identity(3) * 1e9, np.identity(3) * resistance)
load_switch.open()
system_emt.add(load_switch)
system_emt.connect_component(load_switch, [gnd, system_emt.node("n2")])
logger_emt.log_attribute("switchedload_i", "i_intf", load_switch)
load_step_event = dpsimpy.event.SwitchEvent3Ph(0.1 - time_step_emt, load_switch, True)

# Simulation
sim_emt = dpsimpy.Simulation(sim_name_emt, dpsimpy.LogLevel.debug)
sim_emt.set_system(system_emt)
sim_emt.set_time_step(time_step_emt)
sim_emt.set_final_time(final_time_emt)
sim_emt.set_domain(dpsimpy.Domain.EMT)
sim_emt.add_logger(logger_emt)
sim_emt.add_event(load_step_event)
sim_emt.run()

## PF results

In [ ]:
modelName = "EMT_Slack_PiLine_PQLoad_with_PF_Init_PF"
path = "logs/" + modelName + "/"
dpsim_result_file = path + modelName + ".csv"

ts_dpsim_pf = read_timeseries_csv(dpsim_result_file)

## EMT results

In [ ]:
modelName = "EMT_Slack_PiLine_PQLoad_with_PF_Init_EMT"
path = "logs/" + modelName + "/"
dpsim_result_file = path + modelName + ".csv"
PEAK1PH_TO_RMS3PH = np.sqrt(3.0 / 2.0)
ts_dpsim = read_timeseries_csv(dpsim_result_file)

In [ ]:
plt.figure(figsize=(12, 6))
for ts_name, ts_obj in ts_dpsim.items():
    if "_0" in ts_name and "v" in ts_name:
        plt.plot(ts_obj.time, PEAK1PH_TO_RMS3PH * ts_obj.values, label=ts_name)
for ts_name, ts_obj in ts_dpsim_pf.items():
    if "v" in ts_name:
        plt.plot(ts_obj.time, ts_obj.abs().values, label=ts_name + "_pf", linestyle=":")
plt.xlim(4.9, 5.5)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
for ts_name, ts_obj in ts_dpsim.items():
    if "_0" in ts_name and "i" in ts_name:
        plt.plot(ts_obj.time, PEAK1PH_TO_RMS3PH * ts_obj.values, label=ts_name)
for ts_name, ts_obj in ts_dpsim_pf.items():
    if "i" in ts_name:
        plt.plot(ts_obj.time, ts_obj.abs().values, label=ts_name + "_pf", linestyle=":")
plt.xlim(4.9, 5.5)
plt.legend()
plt.show()

In [ ]:
Fs = int(1 / (ts_dpsim["i12_0"].time[1] - ts_dpsim["i12_0"].time[0]))
plt.figure(figsize=(12, 6))
plt.magnitude_spectrum(ts_dpsim["i12_0"].values, Fs=Fs, label="i12_0")

plt.xlim(44, 55)
plt.legend()
plt.show()